In [43]:
from pyspark.sql import SparkSession

import sys
import pandas as pd
from functools import reduce

spark = SparkSession.builder.appName("QueryExecutor").getOrCreate()
sc = spark.sparkContext

In [49]:
queryPath = sys.argv[1] #"./query.sql"
outputFile = sys.argv[2] #"./result.csv"

In [15]:
df = spark.read.csv("./database.csv", header=True)
columnsRenamed = [ (c, c.replace(" ", "")) for c in df.columns]
df = reduce(lambda df, c: df.withColumnRenamed(c[0], c[1]), columnsRenamed, df)
df.createOrReplaceTempView("homicides")

In [36]:
file = open(queryPath, "r")
query = file.read()
file.close()
query = query.replace("\n", " ")
query

'select state, count(1) as count from homicides group by state'

In [40]:
spark.sql(query).toPandas().to_csv(path_or_buf=outputFile)

In [41]:
spark.stop()